In [2]:
### 数据预处理

import pandas as pd
import numpy as np

# import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols

# 加载数据集
titanic = pd.read_csv("data/titanic.csv")

# 数据清洗：填充年龄和票价的缺失值
titanic["age"].fillna(titanic["age"].mean(), inplace=True)
titanic["fare"].fillna(titanic["fare"].median(), inplace=True)

# 查看数据集的前几行
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [10]:
# 单样本t检验
print(titanic['age'].mean().round(2))
t_stat, p_value = stats.ttest_1samp(titanic['age'], 30)
print(f"单样本t检验结果：t统计量={t_stat:.3f}, p值={p_value:.3f}")


29.7
单样本t检验结果：t统计量=nan, p值=nan


1. 单样本t检验的原假设：样本均值等于我们对总体均值的假设
2. 如果p值小于一个给定的显著性水平（比如0.05），那么我们可以“拒绝原假设”，即两者不同。反之则无法拒绝原假设（但不是肯定原假设）
3. p值明显很大，接近0.5，因此我们无法拒绝原假设，即不能拒绝平均年龄是30这个假设。

In [11]:
# 双样本t检验
group1 = titanic[titanic['class'] == 'First']['age']
group3 = titanic[titanic['class'] == 'Third']['age']

print(group1.mean().round(2), group3.mean().round(2))

t_stat, p_value = stats.ttest_ind(group1, group3, equal_var=False)  # 假设方差不等
print(f"双样本t检验结果：t统计量={t_stat:.3f}, p值={p_value:.3f}")


38.23 25.14
双样本t检验结果：t统计量=nan, p值=nan


1. 2独立样本t检验的原假设：2组样本均值相等
2. 如果p值小于一个给定的显著性水平（比如0.05），那么我们可以“拒绝原假设”，即两者不同。反之则无法拒绝原假设（但不是肯定原假设）
3. p值明显很小，接近0，因此我们可以拒绝假设，即认为头等舱和三等舱的乘客年龄有显著差异。

In [12]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
import seaborn as sns

# 加载数据集
titanic = sns.load_dataset('titanic')
titanic['fare'].fillna(titanic['fare'].median(), inplace=True)

# 方差分析
model = ols('fare ~ embark_town', data=titanic).fit()
anova_results = sm.stats.anova_lm(model, typ=2)
print("方差分析结果：")
print(anova_results.round(2))



方差分析结果：
                 sum_sq     df      F  PR(>F)
embark_town   173857.95    2.0  38.14     0.0
Residual     2019361.69  886.0    NaN     NaN


1. 方差分析的原假设：多组样本的均值相等
2. P值很小：拒绝原假设，即最少有1组样本的均值和其他组的均值不同：登船港口的票价有差异

In [9]:
# 创建交叉表
contingency_table = pd.crosstab(titanic['survived'], titanic['sex'])
print("交叉表：")
print(contingency_table)

# 卡方检验
chi2, p_value, dof, expected = stats.chi2_contingency(contingency_table)
print(f"卡方检验结果：卡方统计量={chi2:.3f}, p值={p_value:.3f}, 自由度={dof}, 期望频数：")
print(expected.round(3))


交叉表：
sex       female  male
survived              
0             81   468
1            233   109
卡方检验结果：卡方统计量=260.717, p值=0.000, 自由度=1, 期望频数：
[[193.475 355.525]
 [120.525 221.475]]


1. 卡方检验：2组分类变量独立无关联
2. p值很小：拒绝原假设，即两组变量有关联：性别和生存率是有关系的。